## Earlier steps

1. Convert to plink format  -- mainly need fam file
2. Qiaomei made this, but the individual IDs not population IDs listed in fam.
3. Here, write script to change indIDs to popIDs

In [7]:
##Replace ind with fam
pd="/mnt/solexa/mel_yang/data/HO/"
fh="ind_A_v13_TBH6"

inddict={}
indfile=open(pd+fh+".ind",'r')
for line in indfile:
    x=line.split()
    inddict[x[0]]=x[2]
indfile.close()

famfile=open(pd+fh+".fam",'r')
newfile=open(pd+fh+".pop.fam",'w')
for line in famfile:
    x=line.split()
    newline=line.replace(x[1],inddict[x[1]])
    newfile.write(newline)
famfile.close()
newfile.close()

In [21]:
## Put fam into first column
pd="/mnt/solexa/mel_yang/data/HO/"
fh="ind_A_v13_TBH6"

inddict={}
indfile=open(pd+fh+".ind",'r')
for line in indfile:
    x=line.split()
    inddict[x[0]]=x[2]
indfile.close()
mypops=list(set(inddict.values()))
famfile=open(pd+fh+".fam",'r')
newfile=open(pd+fh+".clust",'w')
num=1
mylst={}
for ind,line in enumerate(famfile):
    x=line.split()
    if inddict[x[1]] in mylst: pass
    else: mylst[inddict[x[1]]]=num; num+=1
    newline="%i\t%s\t%i\n" % (ind+1,x[1],mylst[inddict[x[1]]]) #inddict[x[1]]) #mypops.index(inddict[x[1]]))
    newfile.write(newline)
famfile.close()
newfile.close()

```bash
awk '{print $1,$2,$1}' ind_A_v13_TBH6.pop.fam > ind_A_v13_TBH6.clust
mv ind_A_v13_TBH6.fam ind_A_v13_TBH6.ind.fam
mv ind_A_v13_TBH6.pop.fam ind_A_v13_TBH6.fam
```

In [5]:
##Change SNP file so rsID knows chr_pos
pd="/mnt/solexa/mel_yang/data/HO/"
fh="ind_A_v13_TBH6"

snpfile=open(pd+fh+".snp",'r')
newsnpfile=open(pd+fh+".chrpos.snp",'w')
for line in snpfile:
    x=line.split()
    newid="%s_%s_%s" % (x[1],x[3],x[0])
    newline=line.replace(x[0],newid)
    newsnpfile.write(newline)
newsnpfile.close()
snpfile.close()

In [39]:
##Convert treemix counts to freqs
import gzip
pd="/mnt/solexa/mel_yang/data/HO/"
bgipd="/mnt/solexa/mel_yang/BGI/"
fh="ind_A_v13_TBH6"

inddict={}
indcount={}
indfile=open(pd+fh+".ind",'r')
for line in indfile:
    x=line.split()
    inddict[x[0]]=x[2]
    if x[2] not in indcount: indcount[x[2]]=1
    else: indcount[x[2]]+=1
indfile.close()

cluster={}
clustfile=open(pd+fh+".clust",'r')
for line in clustfile: 
    x=line.split()
    if x[2] not in cluster: cluster[x[2]]=inddict[x[1]]

snps=[0]
snpfile=open(pd+fh+".snp",'r')
for line in snpfile:
    x=line.split()
    snps.append(('chr'+x[1],x[3]))
snpfile.close()


newfreqfile=open(bgipd+fh+".tmix_freqs.txt",'w')
treefile=gzip.open(pd+fh+".treemix_kao.frq.gz",'r')
for ind1,line in enumerate(treefile):
    if ind1==0: 
        header=[cluster[i] for i in line.split()]
        newindfile=open(bgipd+fh+".tmix_nind.txt",'w')
        for i in header: newindfile.write(i+'\t'+str(indcount[i])+'\n')
        newindfile.close()
        newfreqfile.write("#chr\tpos\t%s\n" % '\t'.join(header))
        continue
    x=line.split()
    myden,mynum,myfreqs=[],[],[]
    for strcount in x: 
        lstcount=[float(i) for i in strcount.split(',')]
        myden.append(sum(lstcount))
        mynum.append(lstcount[-1])
    for ind2,i in enumerate(myden):
        if i==0:myfreqs.append("-nan")
        else: myfreqs.append(str(mynum[ind2]/i))
    newfreqfile.write("%s\t%s\t%s\n" % (snps[ind1][0],snps[ind1][1],'\t'.join(myfreqs)))
    #if ind1==50: break
newfreqfile.close()
treefile.close()

In [2]:
%%writefile /mnt/solexa/mel_yang/BGI/checkfreq.py
import numpy as np
import sys

#popiwant="Dai"
popiwant=sys.argv[1]

bgipd="/mnt/solexa/mel_yang/BGI/"
fh1="ind_A_v13_TBH6"

##ORIGINAL
indfile=open(bgipd+fh1+"_nind.txt",'r')
tmixinds1={line.split()[0]:ind for ind,line in enumerate(indfile)}
indfile.close()

myindex1=tmixinds1[popiwant]+2
chrsnps={}
freqs1=[]
freqfile=open(bgipd+fh1+"_freqs.txt",'r')
for ind,line in enumerate(freqfile):
    if line[0]=="#": continue
    x=line.split()
    if x[0] not in chrsnps: chrsnps[x[0]]=[x[1]]
    else: chrsnps[x[0]].append(x[1])
    freqs1.append(float(x[myindex1]))
freqfile.close()

##TREEMIX
indfile=open(bgipd+fh1+".tmix_nind.txt",'r')
tmixinds={line.split()[0]:ind for ind,line in enumerate(indfile)}
indfile.close()

myindex=tmixinds[popiwant]+2

freqs=[]
freqfile=open(bgipd+fh1+".tmix_freqs.txt",'r')
for ind,line in enumerate(freqfile):
    if line[0]=="#": continue
    x=line.split()
    if x[0] not in chrsnps: continue
    if x[1] not in chrsnps[x[0]]: continue

    freqs.append(float(x[myindex]))
freqfile.close()

print len(freqs)
print len(freqs1)
mydiffs=sum([i-freqs1[ind] for ind,i in enumerate(freqs)])
print mydiffs

Overwriting /mnt/solexa/mel_yang/BGI/checkfreq.py
